In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import datetime
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import joblib
import warnings

In [2]:
df = pd.read_csv(r"C:\Users\sachi.mehta\Data Science\Customer segmentation\test2_.csv",encoding = 'unicode_escape')

In [3]:
df_fix = df[df['CustomerID'].notna()]
df_fix["InvoiceDate"] = pd.to_datetime(df_fix["InvoiceDate"],errors='coerce')
df_fix["TotalSum"] = df_fix["Quantity"] * df_fix["UnitPrice"]
snapshot_date = max(df_fix.InvoiceDate) + datetime.timedelta(days=1)
customers = df_fix.groupby(['CustomerID']).agg({'InvoiceDate': lambda x: (snapshot_date - x.max()).days,'InvoiceNo': 'count','TotalSum': 'sum'})
customers.rename(columns = {'InvoiceDate': 'Recency',
                            'InvoiceNo': 'Frequency',
                            'TotalSum': 'MonetaryValue'}, inplace=True)
pd.Series(np.cbrt(customers['MonetaryValue'])).values
customers_fix = pd.DataFrame()
customers_fix["Recency"] = stats.boxcox(customers['Recency'])[0]
customers_fix["Frequency"] = stats.boxcox(customers['Frequency'])[0]
customers_fix["MonetaryValue"] = pd.Series(np.cbrt(customers['MonetaryValue'])).values

scaler = StandardScaler()
scaler.fit(customers_fix)
customers_normalized = scaler.transform(customers_fix)
pd.DataFrame(customers_normalized)
sse = {}
for k in range(1, 3):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(customers_normalized)
    sse[k] = kmeans.inertia_ # SSE to closest cluster centroid

In [4]:
# Load the model from the file 
loaded_model = joblib.load('customer_segmentation.pkl')  
  
# Use the loaded model to make predictions 
y_pred = loaded_model.predict(customers_normalized) 
final = y_pred.tolist()
print(final)

[2, 1, 0]


In [5]:
name = {0:"Active_client", 1:"regular_client",2:"passive_client"}

In [6]:
output = [*map(name.get, final)]
print(output)

['passive_client', 'regular_client', 'Active_client']
